# Quantum Circuit Simulator
### By Matt Wright
#### 2021-02-13

In [78]:
import numpy as np

In [115]:
# Define unitaries:
UNITARIES = {
    'x' : [[0, 1], [1, 0]],
    'h' : [[0.5**0.5, 0.5**0.5], [0.5**0.5, -0.5**0.5]],
    'cx': [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]],
}

def get_ground_state(num_qubits):
    '''Builds a vector representation of the ground state of the QPU.
    
    Parameters
    ----------
    num_qubits : int
        Number of qubits in the desired QPU.
    
    Returns
    -------
    numpy.ndarray
        Vector of length 2**num_qubits with all zero elements except for the first element.
    '''
    
    state_vector = np.zeros(2**num_qubits)
    state_vector[0] = 1
    return state_vector

def get_operator(total_qubits, gate_unitary, target_qubits):
    '''Builds a matrix operator out of the gate or unitary provided to be
    applied to the state vector of the QPU.
    
    Parameters
    ----------
    total_qubits : int
        Number of qubits in the desired QPU.
    gate_unitary : list or str
        This argument is either the unitary matrix or the string representation of the
        unitary (i.e. "h" for Hadamard).
    target_qubits : list[int]
        The qubit indices to apply the operator on.
    
    Returns
    -------
    numpy.ndarray
        Matrix of size 2**total_qubits x 2**total_qubits which can be applied on the
        state vector of the QPU to operate on the target qubit(s).
        
    '''
    
    # Convert to unitary matrix 
    if type(gate_unitary) == list:
        U = gate_unitary
    elif str(gate_unitary) in UNITARIES:
        U = UNITARIES[gate_unitary]
    else:
        raise Exception('Invalid gate/unitary provided.')
    
    if len(target_qubits) == 0:
        raise Exception('No target qubits provided')
    
    elif len(target_qubits) == 1:      
        I = np.identity(2)
        
        if target_qubits[0] == 0:
            operator = U
        else:
            operator = I
            
        for i in range(1, total_qubits):
            if i == target_qubits[0]:
                operator = np.kron(operator, U)
            else:
                operator = np.kron(operator, I)
    else:
        #TODO
        print('not ready yet')
        
    return operator

def run_program(state_vector, program):
    '''Builds and operates the circuit elements on the state vector of the QPU.
    
    Parameters
    ----------
    state_vector : list
        State vector of the QPU to be operated on.
    program : list[dict]
        Defines the desired quantum circuit. Elements are applied in order and should define
        the gate or unitary and the target qubits. For example: `{"unitary": [[0.70710678, 
        0.70710678], [0.70710678, -0.70710678]], "target": [0]}` or `{"gate": "h", "target": [0]}`.
    
    Returns
    -------
    numpy.ndarray
        Updated state vector resulting from its evolution through the circuit.
    '''
    
    if 'unitary' in program[0]:
        gate_unitary_key = 'unitary'
    elif 'gate' in program[0]:
        gate_unitary_key = 'gate'
    else:
        raise Exception('Invalid circuit structure. Must define either "unitary" or "gate" fields in circuit.')
    
    num_qubits = int(np.log2(len(state_vector)))
    for instruction in program:
        O = get_operator(num_qubits, instruction[gate_unitary_key], instruction['target'])
        state_vector = np.dot(O, state_vector)
    return state_vector

def measure_all(state_vector):
    '''Randomly collapses the state vector of the QPU based on the amplitudes of each individual state.
    
    Parameters
    ----------
    state : list
        State vector of the QPU to be measured.
        
    Returns
    -------
    str
        Binary string of the collapsed qubit state in big endian form.
    '''
    
    rand_num = np.random.rand()
    sum_probs = 0
    for idx, val in enumerate(state_vector):
        sum_probs += val**2
        if sum_probs >= rand_num:
            binary_idx = '{0:b}'.format(idx)
            break
    
    num_qubits = int(np.log2(len(state_vector)))
    if len(binary_idx) < num_qubits:
        # Pad binary state with leading zeros if needed
        binary_idx = '0' * (num_qubits-len(binary_idx)) + binary_idx
        
    return binary_idx

def get_counts(state_vector, num_shots=1024):
    '''Measures the state of the QPU multiple times.
    
    Parameters
    ----------
    state_vector : list
        State vector of the QPU to be measured.
    num_shots : int (Default=1024)
        Number of attempts to measure the state.
        
    Returns
    -------
    dict
        Dictionary states that occured and their corresponding counts.
    '''

    if num_shots < 0:
        raise Exception('Invalid `num_shots` provided, must be positive.')
    
    result = {}
    for i in range(num_shots):
        idx = measure_all(state_vector)
        if idx not in result:
            result[idx] = 1
        else:
            result[idx] += 1
    return result

In [116]:
total_qubits = 3

my_circuit = [
    { "gate": "h", "target": [0] }, 
    { "gate": "cx", "target": [0, 1] }
]

# Create "quantum computer" with 2 qubits (this is actually just a vector :) )
my_qpu = get_ground_state(total_qubits)

my_circuit = [
#     { "unitary": [[0.70710678, 0.70710678], [0.70710678, -0.70710678]], "target": [0] }, 
    { "unitary": [[0,1], [1,0]], "target": [2] }, 
#     { "gate": "x", "target": [2] },
]

# Run circuit
final_state = run_program(my_qpu, my_circuit)

print(final_state)

# Read results
counts = get_counts(final_state, 1000)

print(counts)

# Should print something like:
# {
#   "00": 502,
#   "11": 498
# }

''''''

[0. 1. 0. 0. 0. 0. 0. 0.]
{'001': 1000}


''

In [111]:
# TODO
 - Implement CX gate
 - Parameterize circuit
#  - Use the gate letter instead of unitary
 - Add UNITARIES
 - Add capability for higher qubit gates

IndentationError: unexpected indent (<ipython-input-111-0e63ee72626c>, line 2)